In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
sys.path.append(os.path.abspath('../src'))
from data_loader import DiabetesLoader

loader = DiabetesLoader('../data/raw/train.csv')
X_train, y_train, X_val, y_val = loader.get_data()

Loading data from /app/data/raw/train.csv
Data Loaded. Shape: (700000, 25)
Train X: (560000, 24)  Train y: (560000,)
Val X:   (140000, 24)    Val y:   (140000,)


In [3]:
from preprocessing import get_preprocessor

In [4]:
pipeline = get_preprocessor()

In [5]:
print(type(X_train))

<class 'pandas.core.frame.DataFrame'>


In [6]:
import pandas as pd


X_test = pd.read_csv('../data/raw/test.csv')
X_test.head()


,id,age,alcohol_consumption_per_week,physical_activity_minutes_per_week,diet_score,sleep_hours_per_day,screen_time_hours_per_day,bmi,waist_to_hip_ratio,systolic_bp,...,triglycerides,gender,ethnicity,education_level,income_level,smoking_status,employment_status,family_history_diabetes,hypertension_history,cardiovascular_history
0,700000,45,4,100,4.3,6.8,6.2,25.5,0.84,123,...,111,Female,White,Highschool,Middle,Former,Employed,0,0,0
1,700001,35,1,87,3.5,4.6,9.0,28.6,0.88,120,...,145,Female,White,Highschool,Middle,Never,Unemployed,0,0,0
2,700002,45,1,61,7.6,6.8,7.0,28.5,0.94,112,...,184,Male,White,Highschool,Low,Never,Employed,0,0,0
3,700003,55,2,81,7.3,7.3,5.0,26.9,0.91,114,...,128,Male,White,Graduate,Middle,Former,Employed,0,0,0
4,700004,77,2,29,7.3,7.6,8.5,22.0,0.83,131,...,133,Male,White,Graduate,Low,Current,Unemployed,0,0,0


In [7]:
X_train_processed = pipeline.fit_transform(X_train)

In [8]:
X_train_processed.iloc[0]

age                                   52.0
alcohol_consumption_per_week           3.0
physical_activity_minutes_per_week    51.0
diet_score                             2.9
sleep_hours_per_day                    6.1
                                      ... 
employment_status_Student              0.0
employment_status_Unemployed           0.0
family_history_diabetes_1              0.0
hypertension_history_1                 1.0
cardiovascular_history_1               1.0
Name: 363147, Length: 63, dtype: float64

In [9]:
if 'id' in X_test.columns:
    id = X_test.pop('id')

In [10]:
X_val_processed = pipeline.transform(X_val)
X_test_processed = pipeline.transform(X_test)    

In [11]:
ratio = round(sum(y_train == 0)/ sum(y_train == 1), 4)

In [12]:
from xgboost import XGBClassifier

In [13]:
xgb = XGBClassifier(scale_pos_weight=ratio, eval_metric="auc")

xgb.fit(X_train_processed, y_train)

,"objective objective: typing.Union[str, xgboost.sklearn._SklObjWProto, typing.Callable[[typing.Any, typing.Any], typing.Tuple[numpy.ndarray, numpy.ndarray]], NoneType]Specify the learning task and the corresponding learning objective or a customobjective function to be used.For custom objective, see :doc:`/tutorials/custom_metric_obj` and:ref:`custom-obj-metric` for more information, along with the end note forfunction signatures.",'binary:logistic'
,"base_score base_score: typing.Union[float, typing.List[float], NoneType]The initial prediction score of all instances, global bias.",None
,booster,None
,"callbacks callbacks: typing.Optional[typing.List[xgboost.callback.TrainingCallback]]List of callback functions that are applied at end of each iteration.It is possible to use predefined callbacks by using:ref:`Callback API `... note:: States in callback are not preserved during training, which means callback objects can not be reused for multiple training sessions without reinitialization or deepcopy... code-block:: python for params in parameters_grid: # be sure to (re)initialize the callbacks before each run callbacks = [xgb.callback.LearningRateScheduler(custom_rates)] reg = xgboost.XGBRegressor(**params, callbacks=callbacks) reg.fit(X, y)",None
,colsample_bylevel colsample_bylevel: typing.Optional[float]Subsample ratio of columns for each level.,None
,colsample_bynode colsample_bynode: typing.Optional[float]Subsample ratio of columns for each split.,None
,colsample_bytree colsample_bytree: typing.Optional[float]Subsample ratio of columns when constructing each tree.,None
,"device device: typing.Optional[str].. versionadded:: 2.0.0Device ordinal, available options are `cpu`, `cuda`, and `gpu`.",None
,"early_stopping_rounds early_stopping_rounds: typing.Optional[int].. versionadded:: 1.6.0- Activates early stopping. Validation metric needs to improve at least once in every **early_stopping_rounds** round(s) to continue training. Requires at least one item in **eval_set** in :py:meth:`fit`.- If early stopping occurs, the model will have two additional attributes: :py:attr:`best_score` and :py:attr:`best_iteration`. These are used by the :py:meth:`predict` and :py:meth:`apply` methods to determine the optimal number of trees during inference. If users want to access the full model (including trees built after early stopping), they can specify the `iteration_range` in these inference methods. In addition, other utilities like model plotting can also use the entire model.- If you prefer to discard the trees after `best_iteration`, consider using the callback function :py:class:`xgboost.callback.EarlyStopping`.- If there's more than one item in **eval_set**, the last entry will be used for early stopping. If there's more than one metric in **eval_metric**, the last metric will be used for early stopping.",None
,enable_categorical enable_categorical: boolSee the same parameter of :py:class:`DMatrix` for details.,False
,"eval_metric eval_metric: typing.Union[str, typing.List[typing.Union[str, typing.Callable]], typing.Callable, NoneType].. versionadded:: 1.6.0Metric used for monitoring the training result and early stopping. It can be astring or list of strings as names of predefined metric in XGBoost (See:doc:`/parameter`), one of the metrics in :py:mod:`sklearn.metrics`, or anyother user defined metric that looks like `sklearn.metrics`.If custom objective is also provided, then custom metric should implement thecorresponding reverse link function.Unlike the `scoring` parameter commonly used in scikit-learn, when a callableobject is provided, it's assumed to be a cost function and by default XGBoostwill minimize the result during early stopping.For advanced usage on Early stopping like directly choosing to maximize insteadof minimize, see :py:obj:`xgboost.callback.EarlyStopping`.See :doc:`/tutorials/custom_metric_obj` and :ref:`custom-obj-metric` for moreinformation... code-block:: python from sklearn.datasets import load_diabetes 

In [14]:
from sklearn.metrics import  roc_auc_score

y_pred = xgb.predict_proba(X_val_processed)[:,1]

print(roc_auc_score(y_val, y_pred))

0.7210406675860893


In [15]:
importances =pd.DataFrame(xgb.feature_importances_, columns=['importance'], index = X_train_processed.columns.to_list())
display(importances.sort_values(by ='importance',  ascending=False))

,importance
family_history_diabetes_1,0.618076
Relative_Activity,0.071634
Age_BMI_Risk,0.059079
physical_activity_minutes_per_week,0.023322
Chronic_Metabolic_Load,0.016123
...,...
ethnicity_White,0.002179
smoking_status,0.002125
employment_status_Employed,0.001935
gender_Female,0.001881


In [16]:
important_cols = importances[importances.importance > 0].index.to_list()
print(len(important_cols), important_cols)

62 ['age', 'alcohol_consumption_per_week', 'physical_activity_minutes_per_week', 'diet_score', 'sleep_hours_per_day', 'screen_time_hours_per_day', 'bmi', 'waist_to_hip_ratio', 'systolic_bp', 'diastolic_bp', 'heart_rate', 'cholesterol_total', 'hdl_cholesterol', 'ldl_cholesterol', 'triglycerides', 'Pulse_Pressure', 'MAP', 'Rate_Pressure_Product', 'BP_Index', 'Cardio_Stress', 'Non_HDL', 'Trig_HDL_Ratio', 'LDL_HDL_Ratio', 'Cholesterol_Ratio', 'Remnant_Cholesterol', 'Lipid_Accumulation', 'Visceral_Fat_Proxy', 'BS_Index', 'Metabolic_Syndrome_Score', 'Sedentary_Load', 'Diet_Activity_Score', 'Unhealthy_Combo', 'Relative_Activity', 'Activity_Efficiency', 'Alcohol_BMI_Risk', 'Screen_BMI_Interaction', 'Sleep_Stress_Proxy', 'Sleep_Deprivation_Impact', 'Morning_Risk', 'Age_BMI_Risk', 'Age_BP_Risk', 'Age_WHR_Risk', 'Vascular_Aging', 'Chronic_Metabolic_Load', 'education_level', 'income_level', 'smoking_status', 'gender_Female', 'gender_Male', 'gender_Other', 'ethnicity_Asian', 'ethnicity_Black', 'eth

In [17]:
from config import TARGET

final_xgb = XGBClassifier(scale_pos_weight=ratio, eval_metric="auc")

df = pd.read_csv('../data/raw/train.csv')
y_all = df[TARGET]
df.drop(['id',TARGET],axis=1,inplace=True)
X_all_processed = pipeline.transform(df)
X_all_processed = X_all_processed[important_cols]

In [18]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

param_distributions = {
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': randint(3, 8),
    'min_child_weight': randint(1, 10),
    'gamma': uniform(0, 0.5),
    'subsample': uniform(0.6, 0.4),          
    'n_estimators': [100, 200, 300]     
}

xgb_model = XGBClassifier(
    scale_pos_weight=ratio,
    n_jobs = -1, 
    random_state=42
)

random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_distributions,
    n_iter=50,
    scoring='roc_auc',
    cv=3,
    verbose=1,
    n_jobs=1       
)

In [19]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
random_search.fit(X_all_processed, y_all)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


,"estimator estimator: estimator objectAn object of that type is instantiated for each grid point.This is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.","XGBClassifier...ree=None, ...)"
,"param_distributions param_distributions: dict or list of dictsDictionary with parameters names (`str`) as keys and distributionsor lists of parameters to try. Distributions must provide a ``rvs``method for sampling (such as those from scipy.stats.distributions).If a list is given, it is sampled uniformly.If a list of dicts is given, first a dict is sampled uniformly, andthen a parameter is sampled using that dict as above.","{'gamma': <scipy.stats....x7b52579206e0>, 'learning_rate': [0.05, 0.1, ...], 'max_depth': <scipy.stats....x7b525b35ea50>, 'min_child_weight': <scipy.stats....x7b525b35e030>, ...}"
,"n_iter n_iter: int, default=10Number of parameter settings that are sampled. n_iter tradesoff runtime vs quality of the solution.",50
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.If None, the estimator's score method is used.",'roc_auc'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given the ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``RandomizedSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscr

In [ ]:
print(f"Best AUC Score: {random_search.best_score_}")
print(f"Best Parameters: {random_search.best_params_}")
best_random_xgb = random_search.best_estimator_

Best AUC Score: 0.7222835073545085
Best Parameters: {'gamma': np.float64(0.18199832137780103), 'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 300, 'subsample': np.float64(0.9960883942336505)}


In [20]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [4, 5, 6],           
    'min_child_weight': [1, 2],    
    'gamma': [0.1, 0.2],             
    'learning_rate': [0.05, 0.1],     
    'n_estimators': [300, 450],       
    'subsample': [1.0]           
}

grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=3,
    verbose=1,
    n_jobs=1   
)

In [21]:
grid_search.fit(X_all_processed, y_all)

Fitting 3 folds for each of 48 candidates, totalling 144 fits


,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.","XGBClassifier...ree=None, ...)"
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'gamma': [0.1, 0.2], 'learning_rate': [0.05, 0.1], 'max_depth': [4, 5, ...], 'min_child_weight': [1, 2], ...}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'roc_auc'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",3
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candid

In [24]:
best_grid_xgb = grid_search.best_estimator_

In [25]:
print(grid_search.best_params_)

{'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 450, 'subsample': 1.0}


In [26]:
X_test_processed = X_test_processed[important_cols]
y_submit = best_grid_xgb.predict_proba(X_test_processed)[:, 1]

In [27]:
df_submit = pd.DataFrame(y_submit,columns=[TARGET], index=id)

In [28]:
display(df_submit.head())
print(df_submit.shape)

,diagnosed_diabetes
id,
700000,0.407071
700001,0.564679
700002,0.677670
700003,0.302967
700004,0.871406


(300000, 1)


In [ ]:
df_submit.to_csv('../outputs/xgb_feature_engineer_grid_search.csv')

: 